In [1]:
import pandas as pd
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, KFold
from sklearn.pipeline import make_pipeline, Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_validate, cross_val_score
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.datasets import load_iris
import pandas as pd
import numpy as np

In [2]:
iris_data = load_iris()

iris_df = pd.DataFrame(data=iris_data.data, columns=iris_data.feature_names)
iris_df["target"] = iris_data.target

In [3]:
iris_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 150 entries, 0 to 149
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   sepal length (cm)  150 non-null    float64
 1   sepal width (cm)   150 non-null    float64
 2   petal length (cm)  150 non-null    float64
 3   petal width (cm)   150 non-null    float64
 4   target             150 non-null    int64  
dtypes: float64(4), int64(1)
memory usage: 6.0 KB


In [4]:
iris_df.describe()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
count,150.000000,150.000000,150.000000,150.000000,150.000000
mean,5.843333,3.057333,3.758000,1.199333,1.000000
std,0.828066,0.435866,1.765298,0.762238,0.819232
min,4.300000,2.000000,1.000000,0.100000,0.000000
25%,5.100000,2.800000,1.600000,0.300000,0.000000
50%,5.800000,3.000000,4.350000,1.300000,1.000000
75%,6.400000,3.300000,5.100000,1.800000,2.000000
max,7.900000,4.400000,6.900000,2.500000,2.000000


In [5]:
iris_df.head()

,sepal length (cm),sepal width (cm),petal length (cm),petal width (cm),target
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [36]:
# Separate features (X) and target (y)
X = iris_df.drop("target", axis=1)
y = iris_df["target"]

In [37]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.2, random_state=42 )

scaling the features using StandardScaler and then splitting the data into training and validation sets can lead to data leakage in cross-validation. 

To avoid this, you can scale the data within the cross-validation loop using the Pipeline class from scikit-learn.

#### Solution 1: Using Pipeline

In [38]:
# Create a pipeline with scaling and logistic regression
pipe = Pipeline([("scaler", StandardScaler()), ("model", LogisticRegression())])

# Perform cross-validation on the training set
scores = cross_val_score(pipe, X_train, y_train, cv=5)

print("Cross-validation scores:", scores)

# Fit the pipeline on the training set and evaluate on the test set
pipe.fit(X_train, y_train)
print("Test accuracy:", pipe.score(X_test, y_test))

Cross-validation scores: [0.95833333 1.         0.875      1.         0.95833333]
Test accuracy: 1.0


#### Solution 2: Using ColumnTransformer

In [41]:
# Define the scaler and model
scaler = StandardScaler()
model = LogisticRegression()

# Define the column transformer
ct = ColumnTransformer(
    [("scaler", scaler, np.arange(X.shape[1]))], remainder="passthrough"
)

# Define the KFold splitter
kf = KFold(n_splits=5, shuffle=True)

# Perform cross-validation on the training set
scores = []
for train_index, val_index in kf.split(X_train):
    X_train_fold, X_val_fold = X_train.iloc[train_index], X_train.iloc[val_index]
    y_train_fold, y_val_fold = y_train.iloc[train_index], y_train.iloc[val_index]

    # Scale the training data
    X_train_fold_scaled = ct.fit_transform(X_train_fold, y_train_fold)

    # Train the model on the scaled data
    model.fit(X_train_fold_scaled, y_train_fold)

    # Predict on the validation set
    y_pred = model.predict(ct.transform(X_val_fold))

    # Append the scores
    scores.append(model.score(ct.transform(X_val_fold), y_val_fold))

print("Cross-validation scores:", scores)

# Fit the model on the training set and evaluate on the test set
ct.fit(X_train, y_train)
X_train_scaled = ct.transform(X_train)
model.fit(X_train_scaled, y_train)
print("Test accuracy:", model.score(ct.transform(X_test), y_test))

Cross-validation scores: [0.9166666666666666, 0.9583333333333334, 0.9583333333333334, 1.0, 0.9583333333333334]
Test accuracy: 1.0
